In [42]:
import pandas as pd

df = pd.read_csv('merged_relationship.csv', error_bad_lines=False)
#cut -f2 merged_relationship.csv > list1
#cut -f4 merged_relationship.csv > list2

#text_file1 = open('list1_1000', 'r')
text_file1 = open('list1', 'r')
terms1 = text_file1.readlines()
cleaned_terms = [x.rstrip() for x in terms1]

text_file2 = open('list2', 'r')
#text_file2 = open('list2_1000', 'r')
terms2 = text_file2.readlines()
cleaned_terms2 = [x.rstrip() for x in terms2]


#print (cleaned_terms2)


b'Skipping line 91: expected 1 fields, saw 2\nSkipping line 111: expected 1 fields, saw 2\nSkipping line 124: expected 1 fields, saw 2\nSkipping line 201: expected 1 fields, saw 2\nSkipping line 217: expected 1 fields, saw 2\nSkipping line 238: expected 1 fields, saw 3\nSkipping line 270: expected 1 fields, saw 3\nSkipping line 289: expected 1 fields, saw 2\nSkipping line 368: expected 1 fields, saw 2\nSkipping line 370: expected 1 fields, saw 2\nSkipping line 384: expected 1 fields, saw 2\nSkipping line 404: expected 1 fields, saw 2\nSkipping line 406: expected 1 fields, saw 3\nSkipping line 423: expected 1 fields, saw 2\nSkipping line 443: expected 1 fields, saw 3\nSkipping line 451: expected 1 fields, saw 2\nSkipping line 467: expected 1 fields, saw 2\nSkipping line 522: expected 1 fields, saw 2\nSkipping line 550: expected 1 fields, saw 2\nSkipping line 620: expected 1 fields, saw 2\nSkipping line 624: expected 1 fields, saw 2\nSkipping line 808: expected 1 fields, saw 2\nSkipping 

Checking if word tokens from the list are present in wordnet:

In [43]:
import nltk
#nltk.download("wordnet", "/home/natalia/medical_texts/nltk_data/")
nltk.data.path.append('./nltk_data/')

In [44]:
from nltk.corpus import wordnet as wn
wn_lemmas = set(wn.all_lemma_names())
count_present = 0
count_missing = 0

for term in cleaned_terms:
    if term in wn_lemmas:
#        print (term, " is in WordNet")
        #syn_sets = [wn.synsets(cleaned_term) for cleaned_term in cleaned_terms]
        count_present+=1
    else:
#        print (term, " is not in WordNet")
        count_missing+=1

        
print ("In wordNet: ", count_present, "not in wordnet: ", count_missing )

In wordNet:  32348 not in wordnet:  1361


In [45]:

lower_present = 0
lower_missing = 0
lowercased_terms = [x.lower() for x in cleaned_terms]

for term in lowercased_terms:
    if term in wn_lemmas:
#        print (term, " is in WordNet")
        #syn_sets = [wn.synsets(cleaned_term) for cleaned_term in cleaned_terms]
        lower_present+=1
    else:
#        print (term, " is not in WordNet")
        lower_missing+=1
print ("In wordNet: ", lower_present, "not in wordnet: ", lower_missing )    


In wordNet:  32348 not in wordnet:  1361


Many are missing, we need to lemmatize words in the list, mostly a copy-paste from [here](https://rustyonrampage.github.io/text-mining/2017/11/23/stemming-and-lemmatization-with-python-and-nltk.html)

In [46]:
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter 

wnl = WordNetLemmatizer()

lemmatized_terms = []

def get_pos( word ):
    w_synsets = wn.synsets(word)

    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
    most_common_pos_list = pos_counts.most_common(3)
    return most_common_pos_list[0][0] # first indexer for getting the top POS from l

for word in lowercased_terms:
    word.lower()
    lemma = wnl.lemmatize( word, get_pos(word) )
    lemmatized_terms.append(lemma)
 



Now let us search in a lemmatized version of the list (with the help of [this](https://stackoverflow.com/questions/48805672/printing-synsets-of-words-in-a-list-in-python) ):

https://stackoverflow.com/questions/45973596/measuring-similarity-between-different-length-synsets-returns-a-nan

In [ ]:
wn_lemmas = set(wn.all_lemma_names())
count_present = 0
count_missing = 0

def get_synsets(word_term):
    synsets = []
    definitions = []
    #hypernyms = []
    #hyponyms = []
    
    syns = wn.synsets(word_term)
    word = syns[0].lemmas()[0].name()
    definition = syns[0].definition()
    examples = syns[0].examples()
    synonyms = []
    antonyms = []
    
    hypo = lambda s: s.hyponyms()
    hyper = lambda s: s.hypernyms()
    
    
    
    for syn in wn.synsets(word_term):
        hyponyms = list(syn.closure(hypo))
        hypernyms = list(syn.closure(hyper))
        
        
    for lemma in hypernyms:
        #print ("Lemma_name in hypernyms", lemma.name())
        name, pos, sid = lemma.name().split('.')
        #print ("Name: ", name)
        if name == 'causal_agent':
            print ("CAusal agent found!!! for word_term: ", word_term )
        
        for l in syn.lemmas():     
            synonyms.append(l.name())
            if l.antonyms():
                antonyms.append(l.antonyms()[0].name())
    
    
    return definition, examples, set(synonyms), set(antonyms), hyponyms, hypernyms

for term in lemmatized_terms:
    if term in wn_lemmas:
        #print ("============", term, "==========")
        #syn_sets = [wn.synsets(cleaned_term) for cleaned_term in cleaned_terms]
        count_present+=1        
        defs, examples, set_synonyms, set_antonyms, hyponyms, hypernyms = get_synsets(term)
       
        #print ("Synonyms: ", set_synonyms )
        #print ("Hypernyms: ", hypernyms)
        
        
    else:
        #print (term, " is not in WordNet")
        count_missing+=1


        
print ("Lemmatized version is in wordNet: ", count_present, "Lemmatized version is still not in wordnet: ", count_missing )

CAusal agent found!!! for word_term:  donor
CAusal agent found!!! for word_term:  donor
CAusal agent found!!! for word_term:  dosage
CAusal agent found!!! for word_term:  dosage
CAusal agent found!!! for word_term:  dosage
CAusal agent found!!! for word_term:  pi
CAusal agent found!!! for word_term:  pi
CAusal agent found!!! for word_term:  pi
CAusal agent found!!! for word_term:  cigarette
CAusal agent found!!! for word_term:  cigarette
CAusal agent found!!! for word_term:  cigarette
CAusal agent found!!! for word_term:  cigarette
CAusal agent found!!! for word_term:  cigarette
CAusal agent found!!! for word_term:  cigarette
CAusal agent found!!! for word_term:  cigarette
CAusal agent found!!! for word_term:  cigarette
CAusal agent found!!! for word_term:  cigarette
CAusal agent found!!! for word_term:  cigarette
CAusal agent found!!! for word_term:  cigarette
CAusal agent found!!! for word_term:  cigarette
CAusal agent found!!! for word_term:  cigarette
CAusal agent found!!! for word

As for hyponyms and hypernyms, we have to include Synset objects, not just words, see discussion here: 
https://stackoverflow.com/questions/45388056/how-to-remove-synset-and-pos-tag-numbers-on-hypernyms-and-hyponyms
Manual selection of the domain hyponyms/hypernym is thus needed.
Other links: https://stackoverflow.com/questions/13068386/causal-sentences-extraction-using-nltk-python  (causal phrases list)